Türkçe haber ve sentiment sınıflandırmada cosmosGPT, cosmosBERT, BERTurk, trendyol, openchat 3.5 temsillerinin karşılaştırılması için 2 yöntem kullanılacaktır:

1-Temsiller RF, SVM ve LogReg ile modellenecektir. 5 fold cv sonuçlar sunulacaktır.

2-Haber sınıflandırma için “Bu metin “XXX” türündedir. “ ifadesinin temsili ile metnin temsili arasındaki açıya bakılarak sınıflandırılacaktır. Buradaki “XXX” sınıfı ifade eden bir metin olacaktır. Sentiment problemi için “Bu metin olumlu/olumsuz/kötü/iyi/negatif/pozitif ifadeler içermektedir.” “Metinde genel olarak XXX ifadeler yer almaktadır” gibi değişiklikler yapılmalıdır.

5 modelin bu 2 yönteme göre başarıları karşılaştırılacaktır.

Bonus: 2. yöntemde sınıf ifade eden farklı metinlerin başarıya etkisinin karşılaştırılması

## Requirements

In [ ]:
from IPython.display import clear_output
import os

In [ ]:
!pip install datasets
!pip install transformers
!pip install plotly_express
clear_output()

In [ ]:
from datasets import load_dataset #huggingface datasets

#Error: _csv.Error: field larger than field limit (131072)
#Solved: https://stackoverflow.com/questions/15063936/csv-error-field-larger-than-field-limit-131072
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
ds_interpress_news_category_tr_lite_train = load_dataset("interpress_news_category_tr_lite", split="train")
ds_interpress_news_category_tr_lite_test = load_dataset("interpress_news_category_tr_lite", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for interpress_news_category_tr_lite contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/interpress_news_category_tr_lite
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major

Generating train split:   0%|          | 0/218880 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/54721 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for interpress_news_category_tr_lite contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/interpress_news_category_tr_lite
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
print("Number of train news: ", ds_interpress_news_category_tr_lite_train.num_rows)
print("Number of test news: ", ds_interpress_news_category_tr_lite_test.num_rows)

Number of train news:  218880
Number of test news:  54721


In [ ]:
print("Train Features: ", ds_interpress_news_category_tr_lite_train.features)
print("Test Features: ", ds_interpress_news_category_tr_lite_test.features)

Train Features:  {'content': Value(dtype='string', id=None), 'category': ClassLabel(names=['kültürsanat', 'ekonomi', 'siyaset', 'eğitim', 'dünya', 'spor', 'teknoloji', 'magazin', 'sağlık', 'gündem'], id=None)}
Test Features:  {'content': Value(dtype='string', id=None), 'category': ClassLabel(names=['kültürsanat', 'ekonomi', 'siyaset', 'eğitim', 'dünya', 'spor', 'teknoloji', 'magazin', 'sağlık', 'gündem'], id=None)}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import plotly_express as px
import matplotlib.pyplot as plt
import re
import string
import pickle
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

In [ ]:
df_train = pd.DataFrame(ds_interpress_news_category_tr_lite_train)
df_test = pd.DataFrame(ds_interpress_news_category_tr_lite_test)

In [ ]:
df_train

,content,category
0,Tarihten Sınıfta Kaldık Bugün tarihe damgasını...,0
1,"clş ve yatırım ortamı iyileştiriliyor, bürokra...",1
2,Animasyon meraklıları için! Yönetmen: Jennifer...,0
3,İTİ BEEF JJ SVftSL ff 44 BEEFSFISH 16)2016...,0
4,LPG Dolum - Boşaltım kursu KTSO da yapıldı Kar...,1
...,...,...
218875,Begüm-Mert Yazıcıoğiu çiftinin ikinci bebekler...,7
218876,AKRAPOVIC FULL MOON Fiyat açıklanmadı (konsept...,6
218877,Babil gibi bal Notlar Bugün Şenkaya ilçesine b...,0
218878,ÇEVRE İSTATİSTİKLERİNİN ÖZETLERİ YAYINLANDI Tü...,1


In [ ]:
df_train = df_train[:1000]
df_test = df_test[:100]

In [ ]:
df_train

,content,category
0,Tarihten Sınıfta Kaldık Bugün tarihe damgasını...,0
1,"clş ve yatırım ortamı iyileştiriliyor, bürokra...",1
2,Animasyon meraklıları için! Yönetmen: Jennifer...,0
3,İTİ BEEF JJ SVftSL ff 44 BEEFSFISH 16)2016...,0
4,LPG Dolum - Boşaltım kursu KTSO da yapıldı Kar...,1
...,...,...
995,İnternet cilan adları ile ilgili uyuşmazlıklar...,1
996,Çalışan çoğaldıkça harcamalar artıyor Hanede ç...,1
997,rri .T-.ıM#İm Yayın Sahibi Hürriyet Gazetecili...,0
998,Fransız zincirden Almanya da 25 otellik dev al...,1


In [ ]:
def check_data(data):
  null_count = data.isna().sum()
  null_percent = data.isna().sum() / len(data)
  null_percent = null_percent.apply(lambda x: f"{x:.1%}")
  print("Duplicated Data:", data.duplicated().sum())
  result = pd.concat([null_count, null_percent], axis=1)
  result.columns = ['Null Data Count', 'Null Data Percentage']
  return result

In [ ]:
check_data(df_train)

Duplicated Data: 0


,Null Data Count,Null Data Percentage
content,0,0.0%
category,0,0.0%


In [ ]:
df_train.drop_duplicates(inplace=True)

In [ ]:
check_data(df_test)

Duplicated Data: 0


,Null Data Count,Null Data Percentage
content,0,0.0%
category,0,0.0%


In [ ]:
df_test.drop_duplicates(inplace=True)

In [ ]:
_PATH_STOPWORDS = '/content/stopwords_tr_interpress.pkl'
_PATH_BLACKLIST =  '/content/tr-blacklist.pkl'

def get_stopwords():
    try:
        with open(_PATH_STOPWORDS, 'rb') as data_file:
            return pickle.load(data_file)
    except IOError as exc:
        raise IOError("No such stopwords file! Error: " + str(exc))

#  load blacklist
def get_blacklist():
    try:
        with open(_PATH_BLACKLIST, 'rb') as data_file:
            return pickle.load(data_file)
    except IOError as exc:
        raise IOError("No such stopwords file! Error: " + str(exc))


#  cleaning stopwords
def clean_stopwords(content):
    # content: str
    content = content.split(" ")
    filtered_list = []
    stopwords = get_stopwords()
    for word in content:
        if word not in stopwords:
            filtered_list.append(word)

    text = ' '.join(filtered_list)
    return text

#  cleaning blacklist
def clean_blacklist(content):
    # content: str
    # return: str
    content = content.split(" ")
    filtered_list = []
    blacklist = get_blacklist()
    for word in content:
        if word not in blacklist:
            filtered_list.append(word)

    text = ' '.join(filtered_list)
    return text

#  cleaning URLs
def clean_url(content):
    #  content: str
    #  return: str
    reg_url = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    pattern_url = re.compile(reg_url)
    content = pattern_url.sub('',content)
    return content

#  cleaning e-mails
def clean_email(content):
    reg_email='\S*@\S*\s?'
    pattern_email = re.compile(reg_email)
    content = pattern_email.sub('',content)
    return content

#  cleaning punctuation
def clean_punctuation(content):
    #regex = re.compile('[%s]' % re.escape(string.punctuation))
    #content = regex.sub(" ", content)
    content = content.translate(content.maketrans("", "", string.punctuation))
    return content

#  cleaning digits
def clean_numbers(content):
    remove_digits = str.maketrans('', '', string.digits)
    text = content.translate(remove_digits)
    return text

def listToString(text):
    #  text: string
    #  return: string
    str1 = " "
    return (str1.join(text))

#  cleaning postfix
#  for example: Venezuela'nın ==> Venezuela
def clean_postfix(content):
    #  content: list
    #  return: str
    spesific_punctation = [39, 8217] # ascii codes of spesific punctations
    all_words = defaultdict(list)
    counter = 0
    for i, word in enumerate(content):
        if len(word) > 2:
            if chr(8217) in word:
                replaced_word = word.replace(chr(8217), " ")  #  clean for ’
                replaced_word = replaced_word.split(" ")[0]
                all_words[i].append(replaced_word)
            elif chr(39) in word:
                replaced_word = word.replace(chr(39), " ")  # clean for '
                replaced_word = replaced_word.split(" ")[0]
                all_words[i].append(replaced_word)
            else:
                all_words[i].append(word)
    all_words = sorted(all_words.items())

    text = ""
    for i in range(len(all_words)):
        text = text + " " + all_words[i][1][0]

    return text

def clean_text(content):
    #  text: string
    #  return: string
    content = content.lower()
    cleaned_email = clean_email(content)
    cleaned_email_url = clean_url(cleaned_email)
    cleaned_email_url = listToString(cleaned_email_url.split("."))
    cleaned_email_url = cleaned_email_url.split(" ")
    cleaned_email_url_postfix = clean_postfix(cleaned_email_url)
    cleaned_email_url_postfix_punctuation = clean_punctuation(cleaned_email_url_postfix)
    cleaned_email_url_postfix_punctuation_numbers = clean_numbers(cleaned_email_url_postfix_punctuation)
    cleaned_email_url_postfix_punctuation_numbers_stopwords = clean_stopwords(cleaned_email_url_postfix_punctuation_numbers)
    cleaned_email_url_postfix_punctuation_numbers_stopwords_blacklist = clean_blacklist(cleaned_email_url_postfix_punctuation_numbers_stopwords)

    filtered_sentence = []
    for word in cleaned_email_url_postfix_punctuation_numbers_stopwords_blacklist.split(" "):
        if len(word) > 2:
            filtered_sentence.append(word)

    text = ' '.join(filtered_sentence)
    return text


In [ ]:
labels = ["Kültür-Sanat","Ekonomi","Siyaset","Eğitim","Dünya","Spor","Teknoloji","Magazin","Sağlık","Gündem"]

In [ ]:
df_train.content[500]

'SJtv FOX fk r I4 06:30 Kahvaltı Haberleri 08:30 Çocuklar Duymasın 10:00 Müge Anlı İle Tatlı Sert 13:00 DiziTv 13:10 Adanalı 16:00 Esra Erol da Evlen Benimle 19:00 AtvAnaHaber 20:00 Alemin Kiralı 22:30 Tozlu Yollar 07:00Cennet Mahallesi 08 00Hayat Bilgisi 09:30Show Kulüp 1145Saba Tumer ile Bugün 14:15Pis Yedili 17:30Bence Benim Annem 19:00ShowAnaHaber 19:45Spor Sayfası 20:00Pis Yedili 23:30Show Kulüp 06:45 Fatih Portakal ile Çalar Saat 09:45  Serap ile Yeni Bir Umut 12:15 FOX izliyoruz 12:30 Su Gibi 14:45 Unutma Beni 16:30 Yemekteyiz 18:30 Nazlı Tolga ile FOX Ana Haber 1930 DenizYıldızı 20:45 BKM Güldür Güldür İW07:00 08:00 Soru işareti Haber 08:30 08:40 Spor Haber Çizgi Film  Ratz  09:00Laf Arası10:30Yerli Dizi  Eltiler 11:00Haber11:15Kultur Hazinesi12:15Dedektif Lea13:00Haber13:20 13:35 15:00 Spor Haber Dizi  Sarayın Rüzgarı  Haber 15:15 15:30 17:00 Spor Haber Hayatın Tadı Haber 17:15 17:30 18:15 Spor Haber Soru İşareti Umut Var 19:00Ana Haber20:00 20:15 Spor Haber Nılufer de Bu Haft

In [ ]:
clean_text(df_train.content[500])

'sjtv fox kahvaltı çocuklar duymasın müge anlı i̇le tatlı sert dizitv adanalı esra erol evlen benimle atvanahaber alemin kiralı tozlu yollar cennet mahallesi hayat bilgisi show kulüp saba tumer bugün pis yedili bence annem showanahaber spor sayfası pis yedili show kulüp fatih portakal çalar serap umut fox izliyoruz unutma yemekteyiz nazlı tolga fox denizyıldızı bkm güldür güldür i̇w soru işareti spor çizgi film ratz laf arasıyerli dizi eltiler haberkultur hazinesidedektif leahaber spor dizi sarayın rüzgarı spor hayatın tadı spor soru i̇şareti umut spor nılufer renkli çınar görüşler vatandaştan başkana bilimin öncüleri baskı lastv sağlık hattı dünden bugüne hayata dair doğukan sokakta sağlık hattı haberli yazı işleri yaşayan magazin satırbaşı başkent gündemi yapı yaşam alışveriş dünyası hayatadair satırbaşı doğukan sokakta spor trafik buğu bursanın nabzı ajans merkezi sîwbugünbeni affetmelek soframız sevgili dünürümfilm gibien bölümbeni affet star büyük riskmuhteşem yüzyıleyvah düşüyoru

In [ ]:
cleaning = lambda x: clean_text(x)
df_train['content'] = df_train['content'].apply(cleaning)

In [ ]:
df_test['content'] = df_test['content'].apply(cleaning)

In [ ]:
df_train.to_pickle('/content/drive/MyDrive/interpress_news_category_tr_lite_train_cleaned.pkl')
df_test.to_pickle('/content/drive/MyDrive/interpress_news_category_tr_lite_test_cleaned.pkl')

In [ ]:
df_all = pd.concat([df_train, df_test], ignore_index=True)

In [ ]:
df_all

,content,category
0,tarihten sınıfta kaldık bugün tarihe damgasını...,0
1,clş yatırım ortamı iyileştiriliyor bürokrasi a...,1
2,animasyon meraklıları yönetmen jenniferyuh ses...,0
3,i̇ti̇ beef svftsl beefsfish kirmizi eti̇n fran...,0
4,lpg dolum boşaltım kursu ktso yapıldı karaman ...,1
...,...,...
1095,bodrum nutulm bi̇r seyahat gezi̇ tarık gök gez...,0
1096,edi̇töryazdjedibin çabuk geçermiş tezvodafone ...,0
1097,deki kurum kurumsal dönüşüm erp cok sey beklem...,6
1098,tiger vvoods geliyor turki̇sh airlines öpen go...,5


### TF-IDF

In [ ]:
#  word vectorizer and fit
vectorizer_word = TfidfVectorizer(max_features=2000, analyzer='word', ngram_range = (1,3), min_df=3, sublinear_tf=1)
featured_tfidf_word = vectorizer_word.fit(df_all['content'])

#  save word vectorize
with open('/content/drive/MyDrive/interpress_news_category_tr_lite_train_cleaned_tfidf_word_2000.pkl', 'wb') as handle:
    pickle.dump(featured_tfidf_word, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("word feature extracted")

word feature extracted


In [ ]:
#  char vectorizer and fit the data
vectorizer_char = TfidfVectorizer(max_features=2000, analyzer='char', ngram_range = (2,6), sublinear_tf=True, min_df=3)
featured_tfidf_char = vectorizer_char.fit(df_all['content'])
print("char feature extracted")

#  save char vectorizer
with open('/content/drive/MyDrive/interpress_news_category_tr_lite_train_cleaned_tfidf_char_2000.pkl', 'wb') as handle:
    pickle.dump(featured_tfidf_char, handle, protocol=pickle.HIGHEST_PROTOCOL)

char feature extracted


In [ ]:
with open('/content/drive/MyDrive/interpress_news_category_tr_lite_train_cleaned_tfidf_word_2000.pkl', 'rb') as handle:
    word_tfidf = pickle.load(handle)

with open('/content/drive/MyDrive/interpress_news_category_tr_lite_train_cleaned_tfidf_char_2000.pkl', 'rb') as handle:
    char_tfidf = pickle.load(handle)

In [ ]:
def load_data(path):
    with open(path, 'rb') as handle:
        dataset = pickle.load(handle)
    return dataset

In [ ]:
df_train = load_data(r"/content/drive/MyDrive/interpress_news_category_tr_lite_train_cleaned.pkl")
df_test = load_data(r"/content/drive/MyDrive/interpress_news_category_tr_lite_test_cleaned.pkl")

In [ ]:
train_tf_word_transformed = word_tfidf.transform(df_train['content'])
train_tf_char_transformed = char_tfidf.transform(df_train['content'])

test_tf_word_transformed = word_tfidf.transform(df_test['content'])
test_tf_char_transformed = char_tfidf.transform(df_test['content'])

tf_train_features = hstack([train_tf_word_transformed, train_tf_char_transformed])
tf_train_features.shape

(1000, 4000)

In [ ]:
tf_train_features

<1000x4000 sparse matrix of type '<class 'numpy.float64'>'
	with 995067 stored elements in Compressed Sparse Row format>

## Train with Sckit-Learn

In [ ]:
tf_train_features

In [ ]:
# Maybe using here tf.estimator for acceleration ?
models = [
    RandomForestClassifier(),
    SVC(kernel='linear', gamma='auto'),
    LogisticRegression()
]

accuracy = []

for model in models:
    model_scores = cross_val_score(model, tf_train_features, df_train['category'], cv=5)
    accuracy.append({"Model name": type(model).__name__, "Accuracy Scores": model_scores})
    print(f"Accuracy with {type(model).__name__}: {model_scores}")
    print("---" * 15)

In [ ]:
accuracy_df = pd.DataFrame(accuracy)
accuracy_df

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as ps
import pandas as pd
import numpy as np

# Sample data
data = {
    'Model name': ['RandomForestClassifier', 'SVC', 'LogisticRegression'],
    'Accuracy Scores': [
        [0.585, 0.595, 0.57, 0.555, 0.625],
        [0.77, 0.755, 0.785, 0.8, 0.805],
        [0.71, 0.69, 0.72, 0.715, 0.73]
    ]
}

accuracy_df = pd.DataFrame(data)

# Create subplots
fig = ps.make_subplots(rows=len(accuracy_df), cols=1, subplot_titles=accuracy_df['Model name'])

# Add traces
for i, row in accuracy_df.iterrows():
    model_name = row['Model name']
    accuracies = row['Accuracy Scores']
    iterations = np.arange(1, len(accuracies) + 1)

    fig.add_trace(go.Scatter(x=iterations, y=accuracies, mode='lines', name=model_name), row=i+1, col=1)

# Update layout
fig.update_layout(height=600, width=800, title_text="Machine Learning Model Accuracies",
                  xaxis_title="Iteration", yaxis_title="Accuracy Score", template='plotly_dark')

# Update subplot titles
for i, model_name in enumerate(accuracy_df['Model name']):
    fig.update_yaxes(title_text="Accuracy", row=i+1, col=1)

# Show plot
fig.show()


## News classification with angle

In [ ]:
!pip install scikit-learn
clear_output()

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

X_text = df_train['content']

categories = ["Kültür-Sanat","Ekonomi","Siyaset","Eğitim",
              "Dünya","Spor","Teknoloji","Magazin","Sağlık","Gündem"]

XXX_expressions = [f'bu metin {category} türündedir' for category in categories]

vectorizer = TfidfVectorizer()
X_vectors = vectorizer.fit_transform(X_text)
XXX_vectors = vectorizer.transform(XXX_expressions)

angle_measurements = []

for text_vector in X_vectors:
    angles = []
    for expression_vector in XXX_vectors:
        angle = np.arccos(cosine_similarity(expression_vector, text_vector))
        angles.append(angle)
    angle_measurements.append(angles)

predicted_classes = []

threshold = np.pi / 2
for angles in angle_measurements:
    min_angle = min(angles)
    min_index = angles.index(min_angle)
    if min_angle < threshold:
        predicted_class = categories[min_index]
    else:
        predicted_class = "Diğer"
    predicted_classes.append(predicted_class)

In [ ]:
df_predicted = pd.DataFrame({'predicted_class': [f"bu metin {predicted_class} türündedir" for predicted_class in predicted_classes]})

df_train['predicted_class'] = df_predicted['predicted_class']

In [ ]:
df_train

## Sentiment Analysis

### bert-base-turkish-sentiment-cased

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
var_1 = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
p = var_1("bu telefon modelleri çok kaliteli , her parçası çok özel bence")
print(p)
print(p[0]['label'] == 'LABEL_1')

[{'label': 'positive', 'score': 0.983146607875824}]
False


In [ ]:
def sentiment_label(label):
    if label == 'positive':
        return "Bu metin pozitif ifadeler içermektedir"
    else:
        return "Bu metin negatif ifadeler içermektedir"

def analyze_sentiment(df_train):
  """
  This function performs sentiment analysis on each row of a DataFrame.
  """
  df_train['sentiment'] = ''
  for index, row in df_train.iterrows():
    text = row['content']
    sentiment_dict = var_1(text[:512])  # Assuming var_1 performs sentiment analysis
    label = sentiment_dict[0]['label']  # label: positive
    df_train.at[index, 'sentiment'] = sentiment_label(label)
  return df_train

df_train = analyze_sentiment(df_train.copy())  # Avoid modifying original DataFrame

In [ ]:
df_train.to_csv('data_sentiment.csv', index=False)

### cosmosGPT-2

In [ ]:
from transformers import AutoTokenizer, GPT2ForSequenceClassification
from transformers import pipeline

model = GPT2ForSequenceClassification.from_pretrained("ytu-ce-cosmos/turkish-gpt2")
tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-gpt2")

var_2 = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ytu-ce-cosmos/turkish-gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/585k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
p = var_2("bu telefon modelleri çok kaliteli , her parçası çok özel bence")
print(p)
print(p[0]['label'] == 'LABEL_1')

[{'label': 'LABEL_0', 'score': 0.6140037178993225}]
False


In [ ]:
def sentiment_label(label):
    if label == 'LABEL_1':
        return "Bu metin pozitif ifadeler içermektedir"
    else:
        return "Bu metin negatif ifadeler içermektedir"

def analyze_sentiment(df_train):
  """
  This function performs sentiment analysis on each row of a DataFrame.
  """
  df_train['sentiment_gpt2'] = ''
  for index, row in df_train.iterrows():
    text = row['content']
    sentiment_dict = var_2(text[:512])  # Assuming var_2 performs sentiment analysis
    label = sentiment_dict[0]['label']  # label: positive
    df_train.at[index, 'sentiment_gpt2'] = sentiment_label(label)
  return df_train

df_train = analyze_sentiment(df_train.copy())  # Avoid modifying original DataFrame

### cosmosBertUncased

In [ ]:
# Load model directly
from transformers import AutoTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-base-bert-uncased")
model = BertForSequenceClassification.from_pretrained("ytu-ce-cosmos/turkish-base-bert-uncased")
var_3 = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ytu-ce-cosmos/turkish-base-bert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
p = var_3("bu telefon modelleri çok kaliteli , her parçası çok özel bence")
print(p)
print(p[0]['label'] == 'LABEL_1')

[{'label': 'LABEL_1', 'score': 0.639157772064209}]
True


In [ ]:
def sentiment_label(label):
    if label == 'LABEL_1':
        return "Bu metin pozitif ifadeler içermektedir"
    else:
        return "Bu metin negatif ifadeler içermektedir"

def analyze_sentiment(df_train):
  """
  This function performs sentiment analysis on each row of a DataFrame.
  """
  df_train['sentiment_bert-base-uncased'] = ''
  for index, row in df_train.iterrows():
    text = row['content']
    sentiment_dict = var_3(text[:512])  # Assuming var_1 performs sentiment analysis
    label = sentiment_dict[0]['label']  # label: positive
    df_train.at[index, 'sentiment_bert-base-uncased'] = sentiment_label(label)
  return df_train

df_train = analyze_sentiment(df_train.copy())  # Avoid modifying original DataFrame

In [ ]:
df_train

,content,category,sentiment,sentiment_gpt2,sentiment_bert-base-uncased
0,tarihten sınıfta kaldık bugün tarihe damgasını...,0,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
1,clş yatırım ortamı iyileştiriliyor bürokrasi a...,1,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
2,animasyon meraklıları yönetmen jenniferyuh ses...,0,Bu metin pozitif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
3,i̇ti̇ beef svftsl beefsfish kirmizi eti̇n fran...,0,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
4,lpg dolum boşaltım kursu ktso yapıldı karaman ...,1,Bu metin pozitif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
...,...,...,...,...,...
995,i̇nternet cilan adları uyuşmazlıklar internet ...,1,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
996,çalışan çoğaldıkça harcamalar artıyor hanede ç...,1,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
997,rri ımi̇m yayın sahibi hürriyet gazetecilik ma...,0,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
998,fransız zincirden almanya otellik alım yıldır ...,1,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir


### BERTurk

In [ ]:
# Load model directly
from transformers import AutoModel, BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased")
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
var_4 = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

In [ ]:
p = var_4(df_train['content'][0][:512])
print(p)
print(p[0]['label'] == 'LABEL_1')

[{'label': 'LABEL_0', 'score': 0.5260725617408752}]
False


In [ ]:
def sentiment_label(label):
    if label == 'LABEL_1':
        return "Bu metin pozitif ifadeler içermektedir"
    else:
        return "Bu metin negatif ifadeler içermektedir"

def analyze_sentiment(df_train):
  """
  This function performs sentiment analysis on each row of a DataFrame.
  """
  df_train['sentiment_bertTurkish-cased'] = ''
  for index, row in df_train.iterrows():
    text = row['content']
    sentiment_dict = var_3(text[:512])  # Assuming var_1 performs sentiment analysis
    label = sentiment_dict[0]['label']  # label: positive
    df_train.at[index, 'sentiment_bertTurkish-cased'] = sentiment_label(label)
  return df_train

df_train = analyze_sentiment(df_train.copy())  # Avoid modifying original DataFrame

In [ ]:
df_train

,content,category,sentiment,sentiment_gpt2,sentiment_bert-base-uncased,sentiment_bertTurkish-cased
0,tarihten sınıfta kaldık bugün tarihe damgasını...,0,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
1,clş yatırım ortamı iyileştiriliyor bürokrasi a...,1,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
2,animasyon meraklıları yönetmen jenniferyuh ses...,0,Bu metin pozitif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
3,i̇ti̇ beef svftsl beefsfish kirmizi eti̇n fran...,0,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
4,lpg dolum boşaltım kursu ktso yapıldı karaman ...,1,Bu metin pozitif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
...,...,...,...,...,...,...
995,i̇nternet cilan adları uyuşmazlıklar internet ...,1,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
996,çalışan çoğaldıkça harcamalar artıyor hanede ç...,1,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
997,rri ımi̇m yayın sahibi hürriyet gazetecilik ma...,0,Bu metin negatif ifadeler içermektedir,Bu metin negatif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir
998,fransız zincirden almanya otellik alım yıldır ...,1,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir,Bu metin pozitif ifadeler içermektedir


## Residuals

In [ ]:
df_train.columns

Index(['content', 'category', 'sentiment', 'sentiment_gpt2',
       'sentiment_bert-base-uncased', 'sentiment_bertTurkish-cased'],
      dtype='object')

In [ ]:
df_for_residuals = pd.DataFrame()
for index, text in df_train['sentiment'].items():
    if 'pozitif' in text:
        df_for_residuals.loc[index, 'sentiment_label_sentiment_cased'] = True
    else:
        df_for_residuals.loc[index, 'sentiment_label_sentiment_cased'] = False

for index, text in df_train['sentiment_gpt2'].items():
    if 'pozitif' in text:
        df_for_residuals.loc[index, 'sentiment_label_gpt2'] = True
    else:
        df_for_residuals.loc[index, 'sentiment_label_gpt2'] = False

for index, text in df_train['sentiment_bert-base-uncased'].items():
    if 'pozitif' in text:
        df_for_residuals.loc[index, 'sentiment_label_bert-base-uncased'] = True
    else:
        df_for_residuals.loc[index, 'sentiment_label_bert-base-uncased'] = False

for index, text in df_train['sentiment_bertTurkish-cased'].items():
    if 'pozitif' in text:
        df_for_residuals.loc[index, 'sentiment_label_bertTurkish-cased'] = True
    else:
        df_for_residuals.loc[index, 'sentiment_label_bertTurkish-cased'] = False


In [ ]:
df_for_residuals

,sentiment_label_sentiment_cased,sentiment_label_gpt2,sentiment_label_bert-base-uncased,sentiment_label_bertTurkish-cased
0,False,False,True,True
1,False,False,True,True
2,True,False,True,True
3,True,True,True,True
4,True,False,True,True
...,...,...,...,...
995,False,False,True,True
996,False,False,True,True
997,False,False,True,True
998,True,True,True,True


### Bar Plots

In [ ]:
fig = px.bar(df_for_residuals, x='sentiment_label_sentiment_cased', color='sentiment_label_sentiment_cased', title='Distribution of sentiment_label_sentiment_cased')
fig.show()

In [ ]:
fig = px.bar(df_for_residuals, x='sentiment_label_gpt2', color='sentiment_label_gpt2',  title='Distribution of sentiment_label_gpt2')
fig.show()

In [ ]:
fig = px.bar(df_for_residuals, x='sentiment_label_bert-base-uncased', color='sentiment_label_bert-base-uncased',  title='Distribution of sentiment_label_bert-base-uncased')
fig.show()

In [ ]:
fig = px.bar(df_for_residuals, x='sentiment_label_bertTurkish-cased', color='sentiment_label_bertTurkish-cased',  title='Distribution of sentiment_label_bertTurkish-cased')
fig.show()